<a href="https://colab.research.google.com/github/Nishidesai2002/HelloWorld/blob/master/Signature_validation_with_percent_of_white_background.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import cv2
import numpy as np

# Load images from the specified folder
def load_images_from_folder(folder):
    images = []
    image_paths = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        if img_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            img = cv2.imread(img_path)
            if img is not None:
                images.append(img)
                image_paths.append(img_path)
    return images, image_paths

# Path to the folder containing the signatures to validate
folder_to_validate = r"/content/signature"

# Load images from the folder
images, image_paths = load_images_from_folder(folder_to_validate)

# Function to analyze the background of an image
def analyze_background(image):
    # Convert to grayscale and apply a threshold
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY)

    # Calculate the percentage of white background
    background_area = np.sum(thresh == 255)
    total_area = thresh.size
    background_percentage = (background_area / total_area) * 100

    return background_percentage

# Function to detect blurriness in an image
def detect_blurriness(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()
    return laplacian_var

# Function to check for capture imperfections in an image
def check_imperfections(image):
    height, width = image.shape[:2]
    if height < 40 or width < 120:  # Example dimension criteria
        return True
    return False

# Function to validate a signature
def validate_signature(image_path):
    # Load the image
    image = cv2.imread(image_path)

    if image is None:
        print(f"{image_path}: Could not load the image.")
        return {"signature_valid": False, "reason": "Image loading failed"}

    # Analyze the background
    bg_percentage = analyze_background(image)

    # Check for blurriness
    blur_score = detect_blurriness(image)

    # Check for imperfections
    imperfection_flag = check_imperfections(image)

    # Define thresholds
    WHITE_BG_THRESHOLD = 95.0  # Percentage of white background
    BLUR_THRESHOLD = 100.0  # Laplacian variance threshold

    # Validate based on the checks
    if bg_percentage > WHITE_BG_THRESHOLD:
        return {"signature_valid": False, "reason": "Excessive white background"}

    if blur_score < BLUR_THRESHOLD:
        return {"signature_valid": False, "reason": "Blurriness detected"}

    if imperfection_flag:
        return {"signature_valid": False, "reason": "Capture imperfections"}

    return {
        "signature_valid": True,
        "background_percentage": bg_percentage,
        "blur_score": blur_score
    }

# Iterate through all images in the folder and validate them
results = []
for image_path in image_paths:
    result = validate_signature(image_path)
    print(f"{image_path}: {result}")
    results.append(result)

# Print final summary of results
print("\nSummary of Validation Results:")
for i, res in enumerate(results):
    print(f"{image_paths[i]}: {res}")


/content/signature/107_data (2750).jpg: {'signature_valid': True, 'background_percentage': 0.0, 'blur_score': 167.66515361639196}
/content/signature/103_data (3760).jpg: {'signature_valid': True, 'background_percentage': 0.26359932901988975, 'blur_score': 2053.1487138395396}
/content/signature/96_data.jpg: {'signature_valid': False, 'reason': 'Blurriness detected'}
/content/signature/IMG_20240607_103003.jpg: {'signature_valid': False, 'reason': 'Blurriness detected'}
/content/signature/IMG_20240607_103003 (1).jpg: {'signature_valid': False, 'reason': 'Blurriness detected'}
/content/signature/98_data (2986).jpg: {'signature_valid': True, 'background_percentage': 0.0, 'blur_score': 289.9210480862852}

Summary of Validation Results:
/content/signature/107_data (2750).jpg: {'signature_valid': True, 'background_percentage': 0.0, 'blur_score': 167.66515361639196}
/content/signature/103_data (3760).jpg: {'signature_valid': True, 'background_percentage': 0.26359932901988975, 'blur_score': 2053